In [3]:
pip install TTS

In [7]:
!pip install flask torch torchaudio TTS pyngrok flask-cors


In [ ]:
import os
import torch
import torchaudio
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from TTS.api import TTS
from pyngrok import ngrok

# Create directories for uploads and outputs
UPLOAD_FOLDER = "./uploads"
OUTPUT_FOLDER = "./outputs"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

app = Flask(__name__)
CORS(app)  # Enable CORS to allow frontend requests from different origins

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

# Set your ngrok auth token here
ngrok.set_auth_token("2t5jG4Kb85xObYFT6mzyG088vu1_61sQEm5o3GesZbmChuQHR")
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Load TTS model and move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2").to(device)
print("TTS model loaded on", device)

@app.route('/')
def home():
    return jsonify({"message": "Backend is running."})

@app.route('/process', methods=['POST'])
def process():
    # Validate file upload
    if 'voice_sample' not in request.files:
        return jsonify({"error": "No voice sample provided."}), 400
    file = request.files['voice_sample']
    if file.filename == '':
        return jsonify({"error": "No file selected."}), 400

    # Validate text input
    text = request.form.get('text')
    if not text:
        return jsonify({"error": "Text input is required."}), 400

    # Save the uploaded voice sample
    input_filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(input_filepath)

    # Load the audio file and resample if needed
    try:
        waveform, sample_rate = torchaudio.load(input_filepath)
    except Exception as e:
        return jsonify({"error": f"Error loading audio file: {str(e)}"}), 400

    if sample_rate != 22050:
        try:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=22050)
            waveform = resampler(waveform)
        except Exception as e:
            return jsonify({"error": f"Error during resampling: {str(e)}"}), 400

    # Save the cleaned voice file for TTS processing
    cleaned_filename = "cleaned_" + file.filename
    cleaned_filepath = os.path.join(app.config['UPLOAD_FOLDER'], cleaned_filename)
    torchaudio.save(cleaned_filepath, waveform, 22050)

    # Define the output file path
    output_filepath = os.path.join(app.config['OUTPUT_FOLDER'], "output_speech.wav")

    try:
        # Generate speech using the TTS model with the provided voice sample
        tts.tts_to_file(text=text, speaker_wav=cleaned_filepath, language="en", file_path=output_filepath)
    except Exception as e:
        return jsonify({"error": f"Error generating speech: {str(e)}"}), 500

    # Return the generated file as an attachment
    return send_file(output_filepath, as_attachment=True)

if __name__ == '__main__':
    app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://9717-35-197-49-174.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


TTS model loaded on cuda
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:39:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:39:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:40:26] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:41:01] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:42:59] "POST / HTTP/1.1" 405 -


 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']
 > Text splitted to sentences.
['hello there i am priyanshu']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:50:47] "POST /process HTTP/1.1" 200 -


...

 > Processing time: 7.608701467514038
 > Real-time factor: 0.9594420084104477


INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:51:48] "POST /process HTTP/1.1" 200 -


 > Processing time: 7.766062021255493
 > Real-time factor: 0.9555043498832896
 > Text splitted to sentences.
['hello there i am priyanshu soni this is an ai generated voice for content creation.']


INFO:werkzeug:127.0.0.1 - - [07/Mar/2025 04:52:53] "POST /process HTTP/1.1" 200 -


 > Processing time: 8.885574340820312
 > Real-time factor: 0.4340966407332054
